In [9]:
import h5py
with h5py.File('train.h5','r') as ipt:
    trainx = ipt['x'][:]
    trainy = ipt['y'][:]

In [21]:
with h5py.File('test.h5','r') as ipt:
    testx = ipt['x'][:]

In [35]:
import pandas as pd,numpy as np

In [23]:
print(trainx.shape,testx.shape)

(528690, 3) (211403, 3)


In [2]:
import torch
from torch import nn
import torch.optim as optim
import torch.utils.data as Data
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"

In [17]:
torch.cuda.set_device(1)

In [18]:
torch.cuda.current_device()

1

In [20]:
torch.cuda.get_device_properties(1)

_CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=12206MB, multi_processor_count=13)

In [22]:
x_train = torch.from_numpy(trainx).view(-1,3).cuda()
y_train = torch.from_numpy(trainy).view(-1,1).cuda()
x_test = torch.from_numpy(testx).view(-1,3).cuda()
learning_rate=0.001

In [24]:
splitnum = int(trainx.shape[0]*0.7)
batch_size = 1000
train_dataset = Data.TensorDataset(x_train[:splitnum],y_train[:splitnum])
train_loader = Data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=False)
test_dataset = Data.TensorDataset(x_train[splitnum:],y_train[splitnum:])
test_loader = Data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [25]:
class Net(torch.nn.Module):
    def __init__(self, nfeature, out):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(nfeature,nfeature*4)
        self.fc2 = torch.nn.Linear(nfeature*4,nfeature*8)
        self.fc3 = torch.nn.Linear(nfeature*8,nfeature*8)
        self.fc = torch.nn.Linear(nfeature*4,out)
        self.conv1 = torch.nn.Conv1d(1,1,(1,3),padding=(0,1))
        self.fc4 = torch.nn.Linear(nfeature*8,nfeature*4)
    def forward(self, x):
        x = F.relu(self.hidden(x))
        #print(x.shape)
        x = F.relu(self.fc2(x))
        #print(x.shape)
        x = F.relu(self.fc3(x))
        #print(x.shape)
        x = F.relu(self.fc4(x))
        x = self.fc(x)
        #x = self.hidden(x)
        return x
model = Net(3,1).to(device)

In [26]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#criterion = nn.MSELoss()
criterion=nn.MSELoss()

In [29]:
epochs = 64
for epoch in range(epochs):
    model.train()
    for batch,(x,y) in enumerate(train_loader):
        ypred = model(x.float())
        loss = criterion(ypred, y.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    size = len(test_loader.dataset)
    test_loss, correct = 0,0
    with torch.no_grad():
        for x,y in test_loader:
            x = x.float()
            pred = model(x)
            test_loss += criterion(pred,y.float()).item()
            correct += (pred.argmax(1) ==y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss:{test_loss:>8f} \n')

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000037 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000034 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000034 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000034 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000034 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000033 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000034 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000033 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000032 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000032 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000033 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000032 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000032 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000032 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000032 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000032 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000032 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000032 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000032 

Test Error: 
 Accuracy: 0.0%, Avg loss:0.000032 



In [30]:
torch.save(model.state_dict(),'model0.pth')

In [38]:
model.eval()
with torch.no_grad():
    x_test = x_test.float()
    pred0 = model(x_test).cpu().numpy()
realpredict = pd.DataFrame({'id': range(testx.shape[0]),'ret':np.array(pred0).reshape(-1)})
realpredict.to_csv('submit0.csv',index=False)